<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 8. Рекуррентные нейронные сети RNN LSTM GRU
Задание
Данные берем отызывы за лето
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [5]:
import keras.backend as K
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [6]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
df = pd.read_excel('/content/отзывы за лето (1).xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [9]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [10]:
df_train.head()

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08


In [11]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [12]:
df_train.head()

,Rating,Content,Date
0,5,наконецтый исправить чушь снеоргинальный проши...,2017-08-09
1,5,удобно использование,2017-07-27
2,5,отлично,2017-08-08
3,5,класс,2017-07-25
4,5,удобно,2017-07-08


In [13]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values
len(text_corpus_train), text_corpus_train

(13841,
 array(['наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться',
        'удобно использование', 'отлично', ...,
        'мочь скачать ошибка номер 24', 'сбербанк', 'целое отлично'],
       dtype=object))

In [14]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False,
                      split = ' ') 

tokenizer.fit_on_texts(text_corpus_train)

len(tokenizer.index_word), tokenizer.index_word

(10292,
 {1: 'приложение',
  2: 'удобно',
  3: 'работать',
  4: 'удобный',
  5: 'отлично',
  6: 'нравиться',
  7: 'хороший',
  8: 'отличный',
  9: 'телефон',
  10: 'супер',
  11: 'быстро',
  12: 'обновление',
  13: 'пароль',
  14: 'мочь',
  15: 'пользоваться',
  16: 'антивирус',
  17: 'банк',
  18: 'вход',
  19: 'устраивать',
  20: 'сбербанк',
  21: 'раз',
  22: 'прошивка',
  23: 'карта',
  24: 'проблема',
  25: 'рута',
  26: 'программа',
  27: 'ошибка',
  28: 'разработчик',
  29: 'сделать',
  30: 'приходиться',
  31: 'вводить',
  32: 'перевод',
  33: 'счёт',
  34: 'писать',
  35: 'норма',
  36: 'деньга',
  37: 'довольный',
  38: 'около',
  39: 'постоянно',
  40: 'нормально',
  41: 'код',
  42: 'исправить',
  43: 'смс',
  44: 'платёж',
  45: 'понятно',
  46: 'последний',
  47: 'функция',
  48: 'зайти',
  49: 'свой',
  50: 'вылетать',
  51: 'мобильный',
  52: 'стать',
  53: 'шаблон',
  54: 'приходить',
  55: 'возможность',
  56: 'право',
  57: 'делать',
  58: 'иня',
  59: 'проверка',
  

In [15]:
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

sequences_train[:10] 

[[610, 42, 775, 3539, 22, 1, 2, 15],
 [2, 181],
 [5],
 [60],
 [2],
 [127, 1],
 [37, 970, 66, 104, 30, 484, 887],
 [102, 51, 1, 690, 139, 732, 106, 1968, 53, 1969, 1625, 2],
 [463, 206, 147, 128, 99, 20, 111, 1059, 51, 9],
 [26, 1371, 888, 5, 3540, 733]]

In [16]:
word_count = len(tokenizer.index_word) + 1  
training_length = max([len(i.split()) for i in text_corpus_train]) 
word_count, training_length 

(10293, 113)

In [17]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)
X_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ...,  27,  84, 223],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]], dtype=int32)

In [18]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [19]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [20]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [21]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length)) 
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(5))
model.add(Activation('softmax'))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [24]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 11s 432ms/step - loss: 0.6434 - get_f1: 0.7813 - val_loss: 0.7017 - val_get_f1: 0.7611
Epoch 2/20
25/25 [==============================] - 13s 543ms/step - loss: 0.5753 - get_f1: 0.8027 - val_loss: 0.6904 - val_get_f1: 0.7625


In [25]:
score_conv = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_conv[0])
print('Test f1_score:', score_conv[1])

14/14 [==============================] - 2s 112ms/step - loss: 0.6539 - get_f1: 0.7828


Test loss: 0.6538544297218323
Test f1_score: 0.7828316688537598


In [26]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64)) 
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [27]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[get_f1])

In [28]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 8s 249ms/step - loss: 1.1192 - get_f1: 0.5280 - val_loss: 0.9000 - val_get_f1: 0.7155
Epoch 2/20
25/25 [==============================] - 6s 227ms/step - loss: 0.8727 - get_f1: 0.7221 - val_loss: 0.8125 - val_get_f1: 0.7375
CPU times: user 21.4 s, sys: 993 ms, total: 22.4 s
Wall time: 14.1 s


In [29]:
score_simplernn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_simplernn[0])
print('Test f1_score:', score_simplernn[1])

14/14 [==============================] - 1s 73ms/step - loss: 0.7997 - get_f1: 0.7437


Test loss: 0.7996675372123718
Test f1_score: 0.7437463402748108


In [30]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[get_f1])

In [32]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 23s 770ms/step - loss: 1.3230 - get_f1: 0.2694 - val_loss: 0.9535 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 21s 856ms/step - loss: 0.9424 - get_f1: 0.6913 - val_loss: 0.8404 - val_get_f1: 0.7329
CPU times: user 1min 10s, sys: 4.26 s, total: 1min 14s
Wall time: 44.8 s


In [33]:
score_lstm= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_lstm[0])
print('Test f1_score:', score_lstm[1])

14/14 [==============================] - 2s 154ms/step - loss: 0.8223 - get_f1: 0.7400


Test loss: 0.8223291039466858
Test f1_score: 0.7400041818618774


In [34]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64,recurrent_dropout=0.2))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [35]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[get_f1])

In [36]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 22s 662ms/step - loss: 1.3096 - get_f1: 0.2085 - val_loss: 0.9664 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 15s 618ms/step - loss: 0.8911 - get_f1: 0.7171 - val_loss: 0.8005 - val_get_f1: 0.7379
CPU times: user 59.1 s, sys: 3.58 s, total: 1min 2s
Wall time: 38 s


In [37]:
score_gru= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_gru[0])
print('Test f1_score:', score_gru[1])

14/14 [==============================] - 2s 128ms/step - loss: 0.7829 - get_f1: 0.7467


Test loss: 0.7829188108444214
Test f1_score: 0.746746838092804


In [38]:
result_RNN = pd.DataFrame({'model': ['SimpleRNN', 'LSTM', 'GRU'], 'f1_score': [score_simplernn[1], score_lstm[1], score_gru[1]]})
result_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
2,GRU,0.746747
0,SimpleRNN,0.743746
1,LSTM,0.740004


CNN --> RNN

In [39]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2)) 
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [40]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[get_f1])

In [41]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 32s 1s/step - loss: 1.2980 - get_f1: 0.3195 - val_loss: 0.9288 - val_get_f1: 0.7113
Epoch 2/20
25/25 [==============================] - 30s 1s/step - loss: 0.9543 - get_f1: 0.6645 - val_loss: 0.7836 - val_get_f1: 0.7331
CPU times: user 1min 39s, sys: 6.74 s, total: 1min 45s
Wall time: 1min 2s


In [42]:
score_cnnrnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_cnnrnn[0])
print('Test f1_score:', score_cnnrnn[1])

14/14 [==============================] - 4s 296ms/step - loss: 0.7704 - get_f1: 0.7401


Test loss: 0.7703760862350464
Test f1_score: 0.7401372194290161


RNN --> CNN

In [43]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[get_f1])

In [45]:
%%time

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 28s 974ms/step - loss: 1.2992 - get_f1: 0.3141 - val_loss: 1.0615 - val_get_f1: 0.7005
Epoch 2/20
25/25 [==============================] - 24s 951ms/step - loss: 1.0560 - get_f1: 0.6303 - val_loss: 0.9457 - val_get_f1: 0.7005
CPU times: user 1min 25s, sys: 4.38 s, total: 1min 30s
Wall time: 51.8 s


In [46]:
score_rnncnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_rnncnn[0])
print('Test f1_score:', score_rnncnn[1])

14/14 [==============================] - 3s 191ms/step - loss: 0.9179 - get_f1: 0.7151


Test loss: 0.9179053902626038
Test f1_score: 0.7151089310646057


In [47]:
result_CNN_RNN = pd.DataFrame({'model': ['CNN_RNN', 'RNN_CNN'], 'f1_score': [score_cnnrnn[1], score_rnncnn[1]]})
result_CNN_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
0,CNN_RNN,0.740137
1,RNN_CNN,0.715109
